# Lab 1 — Task 1: Kings Battles Network (Game of Thrones)

**Goal**: Build a **directed** co-occurrence network of kings who fought in the same battles.

- **Nodes** = Kings (attackers + defenders)  
- **Edges** = *attacker → defender*  
- **Edge weight** = number of battles between the pair  
- **Edge title/tooltip** = list of battle names (and sizes when available)  
- **Node size (value)** = 1 + number of distinct kings this king has attacked (out-degree + 1)

**Inputs**: `data/game-of-thrones-battles.csv`  
**Output HTML**: `Lab1-task1-net5kings.html`

In [2]:
# If needed (e.g., in Colab or a fresh env), uncomment to install dependencies:
%pip install --quiet pyvis pandas


[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from pyvis.network import Network

# ---- Load data ----
csv_path = "data/game-of-thrones-battles.csv"
battles = pd.read_csv(csv_path)

# Keep only the columns we truly need for the network
df = battles[["name", "attacker_king", "defender_king", "attacker_size", "defender_size"]].dropna(subset=["attacker_king", "defender_king", "name"])

# ---- Compute edge weights (# of battles per attacker->defender) ----
weights = (
    df.groupby(["attacker_king", "defender_king"])
    .size()
    .rename("n_battles")
)

# ---- Compute edge titles: comma-joined unique battle names; include sizes when available ----
def _title_group(g):
    names = []
    for _, row in g.iterrows():
        battle = str(row["name"])
        a_sz = row.get("attacker_size", None)
        d_sz = row.get("defender_size", None)
        # Append sizes only when both are not NaN
        if pd.notna(a_sz) and pd.notna(d_sz):
            names.append(f"{battle} (att:{int(a_sz):,} vs def:{int(d_sz):,})")
        else:
            names.append(battle)
    # de-duplicate while preserving order
    seen = set(); ordered = []
    for n in names:
        if n not in seen:
            seen.add(n); ordered.append(n)
    return ", ".join(ordered)

titles = (
    df.groupby(["attacker_king", "defender_king"])
      .apply(_title_group)
      .rename("title")
)

# Merge weights + titles into a single edge table
edges_tbl = pd.concat([weights, titles], axis=1).reset_index()

# ---- Build nodes (all unique kings) ----
kings = sorted(set(df["attacker_king"]).union(set(df["defender_king"])))

# Node "value": out-degree (unique defenders attacked) + 1
out_adj = (
    edges_tbl.groupby("attacker_king")["defender_king"]
    .apply(lambda s: set(s.tolist()))
    .to_dict()
)
node_values = {k: 1 + len(out_adj.get(k, set())) for k in kings}

# ---- Create the network ----
net = Network(
    height="1000px",
    width="100%",
    bgcolor="#242020",
    font_color="white",
    directed=True,
    cdn_resources="remote",   # ensure CDN JS is used in the HTML
)

# Optional: simple color mapping by out-degree bucket (for visual variety)
def color_for_value(v):
    if v >= 4:
        return "red"
    if v == 3:
        return "purple"
    if v == 2:
        return "orange"
    return "green"

for k in kings:
    v = node_values[k]
    net.add_node(k, label=k, value=v, color=color_for_value(v), shape="dot")

# ---- Add edges with weights + titles ----
for _, r in edges_tbl.iterrows():
    net.add_edge(
        r["attacker_king"],
        r["defender_king"],
        value=int(r["n_battles"]),
        title=str(r["title"]),
        arrows="to",
    )

# (Optional) Physics & interaction options similar to class examples
net.set_options('''
{
  "edges": {
    "smooth": { "enabled": true, "type": "dynamic" }
  },
  "physics": {
    "enabled": true,
    "stabilization": { "enabled": true, "iterations": 1000 }
  },
  "interaction": {
    "dragNodes": true,
    "hideEdgesOnDrag": false,
    "hideNodesOnDrag": false
  }
}
''')

out_html = "Lab1-task1-net5kings.html"
net.write_html(out_html)
print(f"✅ Wrote {out_html}")

✅ Wrote Lab1-task1-net5kings.html


/var/folders/vz/g5ckykx10495r0yjfwjkbkx40000gn/T/ipykernel_99399/322020046.py:39: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_title_group)
